In [2]:
import numpy
from networkx import Graph, gnp_random_graph, convert_node_labels_to_integers, set_node_attributes, connected_components, compose, isolates, spring_layout, draw_networkx, draw_networkx_nodes,draw_networkx_edges,adjacency_matrix
from epydemic import ERNetwork, NetworkGenerator, BondPercolation 
from epyc import LabNotebook, HDF5LabNotebook, Lab
from math import ceil
import random
# import matplotlib
# %matplotlib inline
# %config InlineBackend.figure_format = 'png'
# matplotlib.rcParams['figure.dpi'] = 300
# import matplotlib.pyplot as plt
# import seaborn
# matplotlib.style.use('seaborn')
# seaborn.set_context("notebook", font_scale=0.25)
#plotting.BACKEND = 'matplotlib'

### The Modular Network Class

In [3]:
class ModularNetwork(NetworkGenerator):
    
    N_CLUSTER='modular.clusters'#: Experimental parameter for the number of clusters
    P_MODULARITY='modular.modularity' #: Experimental parameter for the modularity
    
    # Node attributes
    ORIGIN = 'origin'
    def __init__(self, nodes_info, phi_info, params=None, limit=None ):
        super(ModularNetwork, self).__init__(params, limit) 
        self.nodes_info = nodes_info # Nodes number in different clusters
        self.phi_info = phi_info # Probability for edge creation in different clusters
    
    def topology(self):
        return 'ER-modular'
    
    def _generate(self, params):
        ncluster = params[self.N_CLUSTER]
        nodes_info=self.nodes_info
        phi_info = self.phi_info
        pModular = params[self.P_MODULARITY]
        
        if pModular == 0:
            print("ERROR: Connection Probability cannot be 0.")
            return None
        
        n_nodes_all=0
        g=Graph()
        gList=list()
        
        # Create all clusters using gnp_random_graph
        for num in range(len(nodes_info)):
            n_cluster = nodes_info[num]
            phi_cluster = phi_info[num]
            g_cluster = gnp_random_graph(n_cluster, phi_cluster)
            g_cluster = g_cluster.subgraph(max(connected_components(g_cluster), key=len)).copy()
            g_cluster = convert_node_labels_to_integers(g_cluster, first_label=n_nodes_all)
            n_nodes_all += n_cluster

            set_node_attributes(g_cluster, name=self.ORIGIN, values=n_nodes_all)
            g = compose(g, g_cluster)
            gList.append(g_cluster)
            
            
        #Find the cluster with the maximum nodes
        # As the central large cluster
        maximum=0
        for cluster in gList:
            if cluster.number_of_nodes() > maximum:
                g_center = cluster
                maximum = cluster.number_of_nodes()
        gList.remove(g_center)

        
        # Connect edges in clusters to the center cluster
        # 1-E_connect/total_edges = pModular
        total_edges = sum(list(ns.number_of_edges() for ns in gList))
        total_add_num = ceil((1-pModular)*total_edges/pModular)
       
        ns_center = list(g_center.nodes())
        for cluster in gList:
            # choose a random node in the centre
            add_num = ceil(total_add_num*(cluster.number_of_edges()/total_edges))
            m = numpy.random.choice(ns_center,size=add_num)
            
            for node in m:
                # choose a random node from the satellite
                ns_sat = list(cluster.nodes())
                n = numpy.random.choice(ns_sat)  
                
                # add the edge
                g.add_edge(n, node)
                
        return g

### The SIR Monitor Class

(From Prof Simon Dobson in the University of St Andrews)

In [4]:
from epydemic import SIR, ProcessSequence, Monitor, ERNetwork, StochasticDynamics
from epyc import Experiment
class MonitoredSIR(SIR):

    def __init__(self):
        super().__init__()

    def build(self, params):
        '''Build the process, adding additional loci
        to be monitored.

        :param params: the parameters'''
        super().build(params)

        # add loci for the other compartments
        self.trackNodesInCompartment(SIR.SUSCEPTIBLE)
        self.trackNodesInCompartment(SIR.REMOVED)

### Run experiment using parallelLab 
1. We run each experiment 3 times and take average 
2. We use ParallelLab to parallel run experiments in order to save time
3. We save the results and experiments information into a JSON file

In [5]:
# select five values from 0.1 to 0.99 for P_modularity
pModularity = numpy.linspace(0.10, 0.99 , num=5, endpoint=True)

In [6]:
#Run experiments
# Parameters set here

from epyc import ParallelLab, JSONLabNotebook,RepeatedExperiment
pnb = JSONLabNotebook('modular-experiments-000.json') # save results in a json file
plab = ParallelLab(pnb, cores=-1) # run experiments parallel 

plab[ModularNetwork.N_CLUSTER] = 4 # set number of clusters
plab[ModularNetwork.P_MODULARITY] = pModularity # set values for P_modularity

# set disease parameter P_infected, P_infect, and P_remove
plab[SIR.P_INFECTED] = 0.005 
plab[SIR.P_INFECT] = 0.0015 
plab[SIR.P_REMOVE] = 0.002 

# capture results every 5 timesteps
# build a compund process from the disease and the monitor
plab[Monitor.DELTA] = 5
p = ProcessSequence([MonitoredSIR(), Monitor()])

# nodes and density in each cluster
nodes_cluster=[1600,200,100,100]
phi_cluster=[0.05,0.03,0.01,0.03]

# use stochastic simulation to run experiments with maximum time 1000 set
# run experiments three times to address randomness
e = StochasticDynamics(p, g=ModularNetwork(nodes_cluster,phi_cluster))
e.process().setMaximumTime(1000)
n=3
plab.runExperiment(RepeatedExperiment(e, n))
df = plab.dataframe()

### Changing p_infect

Use five different modularity. Change P_infect and see the number of removal.


In [156]:
pModularity = numpy.linspace(0.10, 0.99 , num=5, endpoint=True)

from epyc import ParallelLab, JSONLabNotebook,RepeatedExperiment
pnb = JSONLabNotebook('modular-experiments-10.json')
plab = ParallelLab(pnb)

plab[ModularNetwork.N_CLUSTER] = 4
plab[ModularNetwork.P_MODULARITY] = pModularity

# set disease parameter P_infected, P_infect, and P_remove
plab[SIR.P_INFECTED] = 0.0005
plab[SIR.P_INFECT] =  numpy.linspace(0.00001, 0.0009,num=50)
plab[SIR.P_REMOVE] = 0.002

# capture every 5 timesteps
plab[Monitor.DELTA] = 5
# build a compund process from the disease and the monitor
p = ProcessSequence([MonitoredSIR(), Monitor()])

# run the compound process
nodes_cluster=[1600,200,100,100]
phi_cluster=[0.05,0.03,0.01,0.03]

# use stochastic simulation to run experiments with maximum time 1000 set
# run experiments three times to address randomness
e = StochasticDynamics(p, g=ModularNetwork(nodes_cluster,phi_cluster))
e.process().setMaximumTime(1000)
n=10
plab.runExperiment(RepeatedExperiment(e, n))
df = plab.dataframe()

No zero value for type ('<f8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<f8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<f8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<f8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
No zero value for type ('<i8', (201,)), using 0.0
